In [1]:
import pandas as pd
import numpy as np
import json 
import requests

# Repair channel_df

In [3]:
videos_df = pd.read_csv('../data/processed/df_videos_processed.csv')
videos_df.head()

,video_id,title,published,view_count,like_count,comment_count,duration,definition,tags,default_audio_language,madeforkid,playlist_title,channelTitle,playlist_published,playlist_numvideo
0,H_8ZVRRtiIA,"#1 AI for Good Specialization [Course 1, Week ...",2023-11-17 21:49:15+00:00,1671,34.0,1.0,0 days 00:10:01,hd,NaN,en,False,AI for Good Specialization by DeepLearning.AI,DeepLearningAI,2023-07-20 17:01:28+00:00,37.0
1,bZwkaVnhEo4,"#2 AI for Good Specialization [Course 1, Week ...",2023-07-27 16:54:21+00:00,1299,18.0,1.0,0 days 00:07:12,hd,NaN,en,False,AI for Good Specialization by DeepLearning.AI,DeepLearningAI,2023-07-20 17:01:28+00:00,37.0
2,hkOosfWxBK4,"#3 AI for Good Specialization [Course 1, Week ...",2023-11-17 20:13:19+00:00,584,20.0,2.0,0 days 00:03:47,hd,NaN,en,False,AI for Good Specialization by DeepLearning.AI,DeepLearningAI,2023-07-20 17:01:28+00:00,37.0
3,OIavgUGg7mc,"#4 AI for Good Specialization [Course 1, Week ...",2023-07-27 16:54:56+00:00,619,10.0,0.0,0 days 00:04:49,hd,NaN,en,False,AI for Good Specialization by DeepLearning.AI,DeepLearningAI,2023-07-20 17:01:28+00:00,37.0
4,eZZwySXsj-g,"#5 AI for Good Specialization [Course 1, Week ...",2023-07-27 16:55:12+00:00,579,3.0,0.0,0 days 00:05:36,hd,NaN,en,False,AI for Good Specialization by DeepLearning.AI,DeepLearningAI,2023-07-20 17:01:28+00:00,37.0


In [24]:
channel_df = pd.DataFrame(columns=['channel_name', 'playlist_count', 'video_count', 'subscriber_count', 'view_count', 'comment_count', 'join_date', 'country', 'link'])

for channel in videos_df.channelTitle.unique():
    name = channel
    temp_df = videos_df[videos_df.channelTitle == channel]
    playlist_count = temp_df.playlist_title.nunique()
    video_count = temp_df.video_id.nunique()
    view_count = temp_df.view_count.sum()
    comment_count = temp_df.comment_count.sum()

    channel_df = pd.concat([channel_df, pd.DataFrame([[name, playlist_count, video_count, view_count, comment_count]], columns=['channel_name', 'playlist_count', 'video_count', 'view_count', 'comment_count'])])



Because there are some missing information in the video_df, we need to request them from the YouTube API.

In [33]:
api_key = json.load(open('../data/external/API.json', 'r'))['Nam']
channel_id_nam = json.load(open('../data/external/channel_id_nam.json', 'r'))
channel_id_Phuc = json.load(open('../data/external/channel_id_Phuc.json', 'r'))
channel_ids = {**channel_id_nam, **channel_id_Phuc}

def getChannelInfo(channel_id):
    url = 'https://www.googleapis.com/youtube/v3/channels?part=snippet%2CcontentDetails%2Cstatistics%2CbrandingSettings&id={}&key={}'.format(channel_id, api_key)
    response = requests.get(url)
    data = response.json()
    print(data)

    # Extract the required information
    subscriber_count = data['items'][0]['statistics']['subscriberCount']
    join_date = data['items'][0]['snippet']['publishedAt']
    country = data['items'][0]['snippet'].get('country', 'Not available')  # Some channels might not have a 'country' field
    link = "https://www.youtube.com/channel/" + data['items'][0]['id']

    return subscriber_count, join_date, country, link


In [34]:
channel_ids

{'codebasics': 'UCh9nVJoWXmFb7sLApWGcLPQ',
 'TwoMinutePapers': 'UCbfYPyITQ-7l4upoX8nvctg',
 'TheAiGrid': 'UCbY9xX3_jW5c2fjlZVBI4cg',
 'AINewsOfficial': 'UCDZfn935vwaahDabsfylfIQ',
 'abhishekkrthakur': 'UCBPRJjIWfyNG4X-CRbnv78A',
 'IBMTechnology': 'UCKWaEZ-_VweaEx1j62do_vQ',
 'PROROBOTS': 'UCu8luTDe_Xxd2ahAXsCWX5g',
 'StrataScratch': 'UCnVzApLJE2ljPZSeQylSEyg',
 'DeepLearningAI': 'UCcIXc5mJsHVYTZR1maL5l9w',
 '3Blue1Brown': 'UCYO_jab_esuFRV4b17AJtAw',
 'Joma Tech': 'UCV0qA-eDDICsRR9rPcnG7tw',
 'DataCamp': 'UC79Gv3mYp6zKiSwYemEik9A',
 'CS Dojo': 'UCxX9wt5FWQUAAz4UrysqK9A',
 'sentdex': 'UCfzlCWGWYyIQ0aLC5w48gBQ',
 'StatQuest with Josh Starmer': 'UCtYLUTtgS3k1Fg4y5tAhLbw',
 'Tech With Tim': 'UC4JX40jDee_tINbkjycV4Sg',
 '365 Data Science': 'UCEBpSZhI1X8WaP-kY_2LLcg',
 'Data Professor': 'UCV8e2g4IWQqK71bbzGDEI4Q',
 'Data Science Dojo': 'UCzL_0nIe8B4-7ShhVPfJkgw'}

In [35]:
for channel in channel_df.channel_name:
    channel_id = channel_ids[channel]
    subscriber_count, join_date, country, link = getChannelInfo(channel_id)
    channel_df.loc[channel_df.channel_name == channel, ['subscriber_count', 'join_date', 'country', 'link']] = subscriber_count, join_date, country, link
channel_df

{'kind': 'youtube#channelListResponse', 'etag': 'VKCZCmlBs_30z1xVNJ_SBnHwA4g', 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5}, 'items': [{'kind': 'youtube#channel', 'etag': '7dm15v3MJLUNO8aRzpMVwgRCGQM', 'id': 'UCcIXc5mJsHVYTZR1maL5l9w', 'snippet': {'title': 'DeepLearningAI', 'description': 'Welcome to the official DeepLearning.AI YouTube channel! Here you can find the videos from our Coursera programs on machine learning as well as recorded events.\n\nDeepLearning.AI was founded in 2017 by machine learning and education pioneer Andrew Ng to fill a need for world-class AI education.\n\nDeepLearning.AI has created high-quality AI programs on Coursera that have gained an extensive global following. By providing a platform for education and fostering a tight-knit community, DeepLearning.AI has become the pathway for anyone looking to build an AI career.\n', 'customUrl': '@deeplearningai', 'publishedAt': '2017-08-22T22:27:58Z', 'thumbnails': {'default': {'url': 'https://yt3.ggpht.com

KeyError: nan